<a href="https://colab.research.google.com/github/F7FF/Measurement-Class/blob/main/MeasurementClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Oh boy! I can't wait to start work on my third refactor...

# **SIUnit class**: *Represents any SI derived unit.*
Example: `SIUnit("m/s")`

Class variables:
- `match_warning`; When `True`, `__add__` will print a warning if units are 
mismatched.

Instance variables:
- `exponents`; a 7 long list of integers

Exponents correlate to time, length, mass, current, temp, amount, and intensity.

Methods:
- `__str__`; Returns a short human-readable string representing a unit.
- `fancystr`(maybe); Returns a similar string, but more english-ish. Doesn't work very well.
- `+-, */, etc`; Performs arithmetic functions as you'd expect - returns the unit of the result. Note that raising a number to a fractional power might cause an infinite loop!

# **ErrorFloat class**: *Represents a floating point value with an error.*
Example: `ErrorFloat(20.0, 0.1) #values may also be ints`

Class Variables:
- `method`; Sets the method used to propagate error. Legal values are `"stdev"` (default), `"minmax"`

Instance variables
- `value`; (float)
- `error`; (also float)

Methods:
- `__str__`; Returns a human-readable string, like "0.2±0.04"
- `+-, */, etc`; Operates on the value and propagates the error accordingly.
- `tprime(other)`; Returns the tprime score of two errorfloats. (float)
- `ErrorFloat.average(errorfloats)`; Calculates the average of a large number of values, propagating error.



# **Measurement class**: *Represents a measurement with error and units.*
Example: `Measurement(10, 0.1, 'm')` OR `Measurement(0,0,0, measureobject = ErrorFloat(10, 0.1), unitobject = SIUnit('m')`

Class Variables:
- `autoconvert`; When `True`, creating a measurement with a non-SI unit will automatically convert it to SI. For example, `Measurement(10,1,"ft")` will be converted to metres.

Instance variables:
- `measure` (ErrorFloat)
- `unit` (SIUnit)

# **Other functions**:
- `stdev(list)`; Finds the standard deviation of a list
- `plot2d(matrix)`; Returns a string of a nice box-drawn chart

Unfourtunately, using a three-class system results in a LOT of boilerplate. Oh well! 

In [ ]:
import math

#https://en.wikipedia.org/wiki/SI_derived_unit is useful
#https://physics.stackexchange.com/questions/66109/in-what-order-should-unit-symbols-appear was a little helpful

#apparently a guy did this in 2016 - https://github.com/mdipierro/buckingham/blob/master/buckingham/__init__.py - a little limited and uncommented though


#first: a bunch of linear algebra functions to smooth out the SIUnit class's calculation. I was thinking of making a "vector" class, but I don't think these are general enough for all of linear algebra.
def abssum(a): #absolute sum of vector A
  return sum([abs(x) for x in a])
def addvector(a, b):
  return [a[x] + b[x] for x in range(len(a))]
def subvector(a, b):
  return [a[x] - b[x] for x in range(len(a))]
def mulvector(a, k): #multiply vector by constant
  return [x*k for x in a]
def mulsumvector(*args): #for example, if given ((c1, v1), (c2, v2), (c3, v3)) it will return c1*v1 + c2*v2 + c3*v3 as a vector
  vl = len(args[0][1]) #"vector length", ie the length of the vectors
  output = [0 for x in range(vl)]
  for arg in args:
    output = [output[x] + arg[0] * arg[1][x] for x in range(vl)]
  return output
def vectornotwithinmatrix(vector, matrix): #returns True if a vector cannot be formed by multiplying the vectors in a matrix by integers. returns False if not sure or if it can.
  #first; just check that the matrix has nonzero columns in each of the vector's nonzero places
  for x in range(len(vector)):
    if(not any([matrix[y][x] for y in range(len(matrix))]) and (vector[x] != 0)):
      return True 
  return False
def strexp(value): #formats a value as an exponent. can be int or float
  assert type(value) in (int, float)
  if(value == 1):
    return ""
  if(value % 1 == 0.0):
    output = str(int(value))
  else:
    output = str(value)
  font = {".":"˙","-":"⁻","1":"¹","2":"²","3":"³","4":"⁴","5":"⁵","6":"⁶","7":"⁷","8":"⁸","9":"⁹","0":"⁰",}
  return "".join([font[x] for x in output]) #map all the letters



class SIUnit:
  match_warning = True #Global setting: when True, a message will be printed every time incompatible units are added

  _strcache = { #cache of strings for stringifying. If a unit is not in here, str will calculate it and add it to here for next time! ONLY USED WHEN STR() CALLED
    (0,0,0,0,0,0,0):"",
    (-1,0,0,0,0,0,0):"Hz",
    (3,-2,-1,2,0,0,0):"S",
    #if it gets units wrong, add the unit to this dict!
  }
  _unitnames = { #common known units in order to declare new objects
      #"": [0,0,0,0,0,0,0],
      "":     [0,0,0,0,0,0,0],
      "s":    [1,0,0,0,0,0,0],
      "m":    [0,1,0,0,0,0,0],
      "kg":   [0,0,1,0,0,0,0],
      "A":    [0,0,0,1,0,0,0],
      "K":    [0,0,0,0,1,0,0],
      "mol":  [0,0,0,0,0,1,0],
      "cd":   [0,0,0,0,0,0,1],

      "m/s":  [-1,1,0,0,0,0,0],
      "m/s^2":[-2,1,0,0,0,0,0],
      "Hz":   [-1,0,0,0,0,0,0],
      "N":    [-2,1,1,0,0,0,0],
      "Pa":   [-2,-1,1,0,0,0,0],
      "V":    [-3,2,1,-1,0,0,0],
      "ohm":  [-3,2,1,-2,0,0,0],
      "W":    [-3,2,1,0,0,0,0],
      "J":    [-2,2,1,0,0,0,0],
      "C":    [1,0,0,1,0,0,0],
      "F":    [4,-2,-1,2,0,0,0],
      "kg/m^3":[0,-3,1,0,0,0,0],
      "kg/s": [-1,0,1,0,0,0,0],
  }
  _namedunits = ( #derived units that should be used whenever possible. all of these must also be in _longernames!
      #((exponents), (letter, fullname)),
      ((0, 0, 0, 0, 0, 0, 0),''),
      ((1, 0, 0, 1, 0, 0, 0),'C'),
      ((0, 0, 1, 0, 0, 0, 0),'kg'),
      ((-2, 1, 1, 0, 0, 0, 0),'N'),
      ((-2, -1, 1, 0, 0, 0, 0),'Pa'),
      ((-2, 2, 1, 0, 0, 0, 0),'J'),
      ((-3, 2, 1, 0, 0, 0, 0),'W'),
      ((-3, 2, 1, -1, 0, 0, 0),'V'),
      ((4, -2, -1, 2, 0, 0, 0),'F'),
      ((-3, 2, 1, -2, 0, 0, 0),'Ω'),
      ((1, 0, 0, 0, 0, 0, 0),'s'),
      ((0, 1, 0, 0, 0, 0, 0),'m'),
      ((0, 0, 0, 1, 0, 0, 0),'A'),
      ((0, 0, 0, 0, 1, 0, 0),'K'),
      ((0, 0, 0, 0, 0, 1, 0),'mol'),
      ((0, 0, 0, 0, 0, 0, 1),'cd'),
  )

  _longernames = { #contains the longer full english name of a unit
    "":"",
    "C":"coulomb",
    "kg":"kilogram",
    "N":"newton",
    "Pa":"pascal",
    "J":"joule",
    "W":"watt",
    "V":"volt",
    "F":"farad",
    "Ω":"ohm",
    "s":"second",
    "m":"metre",
    "A":"ampere",
    "K":"kelvin",
    "mol":"mole",
    "cd":"candela",
  }

  def __init__(self, foo): #foo can either be a string or a list vector
    if(type(foo) == str):
      if(foo in SIUnit._unitnames):
        self.exponents = SIUnit._unitnames[foo]
      else:
        raise Exception("Unknown unit '" + foo + "'! Try looking at SIUnit._unitnames and Measurement._badunits, or define it manually using a vector of exponents.")
    if(type(foo) == list):
      assert len(foo) == 7 #didya mess up the unit declaration?
      self.exponents = foo #[time (s), length (m), mass (kg), current (A), temperature (K), amount (mol), intensity (cd)]
  def __str__(self):
    if(tuple(self.exponents) in SIUnit._strcache): #check the cache first
      return SIUnit._strcache[tuple(self.exponents)]
    output = ""
    factors = sorted(SIUnit._factorunit(self), key=lambda x:-x[1])
    for factor in factors:
      output += factor[0] + strexp(factor[1])
    SIUnit._strcache[tuple(self.exponents)] = output #add this new unit to the cache for later
    return output
  def fancystr(self): #str but in longer english. kinda messed up but ok
    if(not any(self.exponents)): #if blank
      return ""
    fancyprefixes = {2:"square ", 3:"cubic ", 1:""}
    factors = sorted(SIUnit._factorunit(self), key=lambda x:-x[1])
    if(factors[0][1] < 0): #if the first factor is negative...
      output = "" #"inverse "
    else:
      output = ""
      for pfactor in factors: #ONLY THE POSITIVE ONES
        if(pfactor[1] < 0): break
        if(abs(pfactor[1]) in fancyprefixes):
          output += ' ' + fancyprefixes[abs(pfactor[1])] + SIUnit._longernames[pfactor[0]]
        else:
          output += ' ' + SIUnit._longernames[pfactor[0]] + "^" + str(pfactor[1])
      output += "s"
    for nfactor in factors: #ONLY THE NEGATIVE ONES NOW
      if(nfactor[1] > 0): continue #skip positive ones
      output += " per"
      if(abs(nfactor[1]) in fancyprefixes):
        output += ' ' + fancyprefixes[abs(nfactor[1])] + SIUnit._longernames[nfactor[0]]
      else:
        output += ' ' + SIUnit._longernames[nfactor[0]] + "^" + str(nfactor[1])
    return output
    
    
  def _factorunit(self): #accepts a unit, returns a list like [['A': 4], ['kg': 3], ['m': 2], ['s': 1]] representing how it should be stringified
    acc = self.exponents #an accumulator just for working with
    output = []
    while any(acc): #for as long as there are unfactored units...
      bestunit, bestscore = (), float("inf")
      for namedunit in SIUnit._namedunits:
        for testexponent in range(-10, 10): #just test random exponents lol
          if(abssum(subvector(acc, mulvector(namedunit[0], testexponent))) < bestscore): #if subtracting namedunit^testexponent is better than the best known factor...
            bestscore = abssum(subvector(acc, mulvector(namedunit[0], testexponent)))
            bestunit = (namedunit[1], testexponent, namedunit[0]) #save for if we wanna add to acc later
      #once done trying each possible factor, sub the best one
      output.append((bestunit[0], bestunit[1]))
      acc = subvector(acc, mulvector(bestunit[2], bestunit[1]))
    return output


  def __mul__(self, other):
    return SIUnit(addvector(self.exponents, other.exponents))
  def __truediv__(self, other):
    return self * other.reciprocal()
  def __bool__(self): #false if all exponents 0
    return any(self.exponents)
  def __add__(self, other):
    if(self == other):
      return self
    if(SIUnit._match_warning):
      print("(!!! SIUnit warning: add/sub called on incompatible units " + str(self) + " and " + str(other) + "! By default, " + str(self) + " (left side) will be used. Use 'SIUnit._match_warning = False' to disable warning.)")
    return self
  def __sub__(self, other):
    return self + other #units propagate the same for addition or subtraction (ie. they don't lmao)
  def __eq__(self, other):
    if(type(other) != SIUnit):
      return False
    return self.exponents == other.exponents
  def __pow__(self, other): #oh boy
    return SIUnit(mulvector(self.exponents, int(other)))
  def reciprocal(self):
    return SIUnit([-x for x in self.exponents])


class ErrorFloat:
  method = "stdev" #default propagation method
  trimdigits = True #will trim digits to
  def __init__(self, initvalue, initerror):
    self.value = float(initvalue)
    self.error = abs(float(initerror))
  def __str__(self): #consider padding the value with zeroes until it's the same length as the error!
    if ErrorFloat.trimdigits:
      if(self.error > 10): 
        return str(int(self.value)) + " ±" + str(int(self.error))
      if(self.error == 0):
        return str(self.value) + " ±0"
      x = "{:." + str(-math.floor(math.log10(self.error)) + 1) + "f}"
      return (x + " ±" + x).format(self.value, self.error)
    else:
      return str(self.value) + " ±" + str(self.error)
  def __repr__(self):
    return "ErrorFloat(" + str(self.value) + ", " + str(self.error) + ")"
  def __float__(self):
    return float(self.value) #just ignore the error lmao
  def __int__(self):
    return int(self.value)

  #private methods to make things nicer:
  def _mulerror(a, b): #used when multiplying and dividing
    return ((a.error/a.value)**2 + (b.error/b.value)**2)**0.5
  def _iscon(foo): #returns True if foo doesn't have error and thus is constant
    return type(foo) in (int, float)
  def _frompoints(floats): #when given a list of floats, returns the most precise ErrorFloat that contains all values. eg. [1,2,2.5,3] would return 2 +- 1
    return ErrorFloat.frombounds(min(floats), max(floats))

  #arithmetic dunder methods:
  def __neg__(self):
    return ErrorFloat(-self.value, self.error) #this is true for all propagation methods... I think?
  def __bool__(self):
    return bool(self.value)
  def __add__(self, other):
    if ErrorFloat._iscon(other):
      return ErrorFloat(self.value + other, self.error)
    if(ErrorFloat.method == "stdev"): #add using standard deviation method
      return ErrorFloat(self.value + other.value, (self.error**2 + other.error**2)**0.5)
    if(ErrorFloat.method == "minmax"):
      return ErrorFloat(self.value + other.value, self.error + other.error)
  def __radd__(self, other):
    return self + other
  def __sub__(self, other):
    return self + (-other) #lol
  def __rsub__(self, other):
    return self + (-other)
  def __mul__(self, other):
    if ErrorFloat._iscon(other):
      return ErrorFloat(self.value * other, self.error * other)
    
    if(ErrorFloat.method == "stdev"):
      return ErrorFloat(self.value * other.value, ErrorFloat._mulerror(self, other) * (self.value * other.value))

    if(ErrorFloat.method == "minmax"):
      return ErrorFloat._frompoints([(self.value+self.error)*(other.value+other.error),
                                     (self.value+self.error)*(other.value-other.error),
                                     (self.value-self.error)*(other.value+other.error),
                                     (self.value-self.error)*(other.value-other.error)]) #test each possible combination of 

  def __rmul__(self, other):
    return self * other
  def __truediv__(self, other):
    if ErrorFloat._iscon(other):
      return ErrorFloat(self.value / other, self.error / other)
    return self * other.reciprocal()
  def __rtruediv__(self, other):
    return self.reciprocal() * other
  def __pow__(self, other):
    if ErrorFloat._iscon(other):
      return ErrorFloat(self.value ** other, other * (self.error/self.value))
    if(ErrorFloat.method == "stdev"):
      return ErrorFloat(self.value ** other.value, abs(self.value ** other.value) * ((other.value/self.value * self.error)**2 + (math.log(self.value) * other.error)**2)**0.5) #is this correct?
  def __eq__(self, other): #two errorfloats are equal if their values are the same
    if(type(self) != type(other)):
      return False
    return float(self) == float(other)
  def __ne__(self, other):
    return not (self == other)
  def __lt__(self, other):
    return float(self) < float(other)
  def __gt__(self, other):
    return float(self) > float(other)
  def __le__(self, other):
    return float(self) <= float(other)
  def __ge__(self, other):
    return float(self) >= float(other)

  
  #useful tools
  def average(values): #averages an iterable of errorfloats and propagates error
    sum = ErrorFloat(0,0)
    count = 0
    for value in values:
      sum = sum + value
      count += 1
    return sum / count
  def frombounds(a, b): #returns an errorfloat from two floats - for example, 2 and 3 would return 2.5 +/- 0.5
    return ErrorFloat((a + b) / 2, abs(a - b) / 2)
  def reciprocal(self):
    if(ErrorFloat.method == "stdev"):
      return ErrorFloat(1 / self.value, abs(self.error/self.value) * (1 / self.value)) 
    if(ErrorFloat.method == "minmax"):
      return ErrorFloat.frombounds(1 / (self.value + self.error), 1 / (self.value - self.error)) 
  def tprime(self, other):
    return abs(self.value-other.value)/((self.error**2+other.error**2)**0.5)
  def inrange(self, num):
    return (self.value - self.error < num) and (self.value + self.error > num)

class Measurement:
  autoconvert = True #when True, __init__ will try to convert to SI units immediately if a non-SI unit is given

  _badunits = { #bad units that need to get converted to SI before being initialized as a measurement
      #"badunit":("goodunit", value converter function),
      #"":("", lambda x:x),
      "degC":("K", lambda x:x+273.15),
      "degF":("K", lambda x:(x * (5/9)) + 459.67),

      "lb":("kg", lambda x:x/2.205),
      "lbf":("N", lambda x:x*4.448),

      "yd":("m", lambda x:x*0.9144),
      "ft":("m", lambda x:x/3.281),
      "in":("m", lambda x:x/39.37),
      "mi":("m", lambda x:x*1609.344),

      "km":("m", lambda x:x*1000),
      "cm":("m", lambda x:x/100),
      "mm":("m", lambda x:x/1000),

      "psi":("Pa", lambda x:x*6894.76),
      "bar":("Pa", lambda x:x*100000),
      "atm":("Pa", lambda x:x*101325),
      "torr":("Pa", lambda x:x*133.322), 

      "km/h":("m/s", lambda x:x*0.277778),
      "mph":("m/s", lambda x:x*0.447040357632),
      "fps":("m/s", lambda x:x*0.30480024384),
      "knot":("m/s", lambda x:x*0.514444855556),

      "kJ":("J", lambda x:x*1000),
      "Wh":("J", lambda x:x*3600),
      "kWh":("J", lambda x:x*3600000),
      "eV":("J", lambda x:x * 1.6022 * 10**-19),

      "hp":("W", lambda x:x*745.7),

      "kg/L":("kg/m^3", lambda x:x*1000),
  }

  def __init__(self, initvalue, initerror, initunit, measureobject = None, unitobject = None):
    if(initunit in Measurement._badunits and Measurement.autoconvert): #if it's a bad unit...
        self.unit = SIUnit(Measurement._badunits[initunit][0])
        self.measure = Measurement._badunits[initunit][1](ErrorFloat(initvalue, initerror)) #make an errorfloat using the values and then pass it through whatever function is in _badunits
        return
    if(measureobject != None): #if measureobject is manually defined
      self.measure = measureobject
    else:
      self.measure = ErrorFloat(initvalue, initerror)
    if(unitobject != None): #if unitobject is manually defined
      self.unit = unitobject
    else:
      self.unit = SIUnit(initunit)
  def __float__(self):
    return float(self.measure)
  def __int__(self):
    return int(self.measure)
  def __str__(self):
    return str(self.measure) + " " + str(self.unit)
  def fancystr(self):
    return str(self.measure) + " " + self.unit.fancystr()
  def __neg__(self):
    return Measurement(None, None, None, measureobject = -self.measure, unitobject = self.unit)
  def __bool__(self):
    return bool(self.value)
  def __add__(self, other):
    if ErrorFloat._iscon(other):
      return Measurement(None, None, None, measureobject = self.measure + other, unitobject = self.unit)
    return Measurement(None, None, None, measureobject = self.measure + other.measure, unitobject = self.unit + other.unit)
  def __radd__(self, other):
    return self + other
  def __sub__(self, other):
    return self + (-other)
  def __rsub__(self, other):
    return self + (-other)
  def __mul__(self, other):
    if ErrorFloat._iscon(other):
      return Measurement(None, None, None, measureobject = self.measure * other, unitobject = self.unit)
    return Measurement(None, None, None, measureobject = self.measure * other.measure, unitobject = self.unit * other.unit)
  def __rmul__(self, other):
    return self * other
  def __truediv__(self, other):
    if ErrorFloat._iscon(other):
      return Measurement(None, None, None, measureobject = self.measure / other, unitobject = self.unit)
    return Measurement(None, None, None, measureobject = self.measure / other.measure, unitobject = self.unit / other.unit)
  def __rtruediv__(self, other):
    return self.reciprocal() * other
  def __pow__(self, other):
    if ErrorFloat._iscon(other):
      return Measurement(None, None, None, measureobject = self.measure ** other, unitobject = self.unit ** other)
    return Measurement(None, None, None, measureobject = self.measure ** other.measure, unitobject = self.unit ** other)
  def __eq__(self, other):
    return self.measure == other.measure and self.unit == other.unit
  def __ne__(self, other):
    return not (self == other)
  def __lt__(self, other):
    return float(self) < float(other)
  def __gt__(self, other):
    return float(self) > float(other)
  def __le__(self, other):
    return float(self) <= float(other)
  def __ge__(self, other):
    return float(self) >= float(other)
  #public class methods
  def reciprocal(self):
    return Measurement(None, None, None, measureobject = self.measure.reciprocal(), unitobject = self.unit.reciprocal())
  def inrange(self, num): #returns True if num is inside the tolerances of self
    return self.measure.inrange(num)
  def tprime(self, other):
    return self.measure.tprime(other.measure)


In [ ]:
#some more functions, just to be useful

def plot2d(chart, ylabels=None, xlabels=None, name=None): #just useful for making tables. remember: columns first, then rows!
  def padrow(widths, t): #just a quick function that returns a padding row
      return "┌├└"[t] + '┬┼┴'[t].join(["─" * columnwidth for columnwidth in columnwidths]) + "┐┤┘"[t] + "\n"
  x = len(chart) #getting the x and y widths of the chart
  y = len(chart[0])
  #if(ylabels != None): #if ylabels have been specified
  strchart = [[str(x) for x in row] for row in chart] #make everything a string
  columnwidths = [max([len(y) for y in column]) for column in strchart] #width of the longest column
  strrows = [] #list of strings. each string represents a data row!
  for ycor in range(y):
    strrows.append('│' + '│'.join([strchart[xcor][ycor].rjust(columnwidths[xcor]) for xcor in range(x)]) + '│\n')
  return padrow(columnwidths, 0) + padrow(columnwidths, 1).join(strrows) + padrow(columnwidths, 2) #return toprow + middrow

def rotate(matrix):
  return [[matrix[y][x] for y in range(len(matrix[0]))] for x in range(len(matrix))]
  
def pp(thing): #pass print - ie it prints but also passes through
  print(thing)
  return thing
  
def stdev(values): #finds the standard deviation of a group of values
  average = sum(values) / len(values)
  return ((sum([(x - average)**2 for x in values])) / (len(values) - 1))**0.5

...Finally, we can do some tests to demonstrate its usefulness!

In [ ]:
def test1(): #Uses Measurements to answer a question: how deeply will a hollow open-topped concrete barge float?
  bw = Measurement(20, 0.1, "m") #barge width. all of these are outer dimensions!
  bl = Measurement(65, 0.1, "m") #barge length
  bd = Measurement(10, 0.1, "m") #barge draft (height from bottom)

  densityconcrete = Measurement(2400, 10, "kg/m^3")
  densitywater = Measurement(1000, 50, "kg/m^3")

  wt = Measurement(0.1, 0.01, "m") #wall thickness
  g = Measurement(9.8, 0, "m/s^2") #gravity strength

  #first, calculate the outer and inner volume
  outervolume = bw * bl * bd
  print("Outer volume: ", outervolume)

  innervolume = (bw - 2*wt) * (bl - 2*wt) * (bd - wt) #note that we can include constants freely! we only subtract one wt from bd because the barge is open topped
  print("Inner volume: ", innervolume)

  concretevolume = outervolume - innervolume
  print("Volume of concrete: ", concretevolume)

  #now, use the density of concrete to get the mass of it. units are propagated!
  weight = concretevolume * densityconcrete * g #note that the units will be properly propagated - kg/m^3 * m^3 * m/s^2 cancels out to kgms^-2, which is the definition of a newton of force
  print("Weight: ", weight)

  #now, calculate the volume of water needed to float the barge and divide that by the cross sectional area of the barge
  waterheight = (weight / densitywater / g) / (bw * bl)
  print("Waterheight: ", waterheight)

  tprime = waterheight.tprime(bd) #represents how many error bars of distance are between the two values
  if(tprime > 1):
    print("The barge will definitely float.")
  elif(tprime > 0):
    print("The barge might float, depending on error.")
  else:
    print("The barge will not float.")
  
  print("Pressure at the bottom:", waterheight * densitywater * g) #note that the program independently figured out that distance * density * acceleration = pressure!



test1()

Outer volume:  13000 ±146 m³
Inner volume:  12702 ±145 m³
Volume of concrete:  297 ±206 m³
Weight:  7006702 ±4867718 N
Waterheight:  0.55 ±0.38 m
The barge will definitely float.
Pressure at the bottom: 5389 ±3763 Pa


In [ ]:
def test2(): #answering another question: how quickly can a 20hp elevator lift around 300lbs of people up 200ft! 
  dh = Measurement(200, 1, "ft") #delta height
  power = Measurement(20, 0.1, "hp") #motor power
  g = Measurement(9.8, 0.1, "m/s^2") #gravity strength. just for fun, let's model the fact that gravity is different in different places!
  mass = Measurement(300, 100, "lb") #note! this class assumes you mean pound-weight, not pound-mass. use lbf to specify weight!

  spm = mass * g / power #note - these units are SECONDS PER METER
  print("Seconds per Metre: ", spm) 
  print("Speed: ", 1/spm) #there we go!
  print("Time: ", dh * spm)

test2()

Seconds per Metre:  0.089 ±0.030 sm⁻¹
Speed:  11.2 ±3.7 ms⁻¹
Time:  5.4 ±1.8 s


In [ ]:
def test3(): #performs some calculations based on fluid flow in some pipes - the pipes start with radius r1 and narrow down to r2 at the end. 
  flow = Measurement(1, 0.1, "", unitobject = SIUnit("m")**3 * SIUnit("s")**-1 ) #m^3/s isn't a unit it knows, so I must initialize using a small formula
  r1 = Measurement(10, 0.1, "cm")

  widths = ["Widths:"]
  areas = ["Cross sectional areas:"]
  chart = ["Speeds:"]
  for x in range(1, 10):
    r2 = Measurement(x/10, 0.001, "m")
    widths.append(r2)
    areas.append(3.141 * r2**2)
    chart.append(flow / (3.141 * r2**2))
  
  print(plot2d((widths, areas, chart)))

test3()

┌────────────────┬──────────────────────┬─────────────────┐
│         Widths:│Cross sectional areas:│          Speeds:│
├────────────────┼──────────────────────┼─────────────────┤
│0.1000 ±0.0010 m│       0.031 ±0.063 m²│      31 ±63 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.2000 ±0.0010 m│       0.126 ±0.031 m²│    8.0 ±2.1 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.3000 ±0.0010 m│       0.283 ±0.021 m²│  3.54 ±0.44 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.4000 ±0.0010 m│       0.503 ±0.016 m²│  1.99 ±0.21 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.5000 ±0.0010 m│       0.785 ±0.013 m²│  1.27 ±0.13 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.6000 ±0.0010 m│       1.131 ±0.010 m²│0.884 ±0.089 ms⁻¹│
├────────────────┼──────────────────────┼─────────────────┤
│0.7000 ±0.0010 m│     1.5391 ±0.0090 m²│0.650 ±0.065 ms⁻¹│
├────────────────┼──────────────────────

In [ ]:
def test4(): #calculates the voltage across an electric component
  ohm = Measurement(220, 22, "ohm")
  amperage = Measurement(0.13, 0, "A")

  print(">Voltage: ", ohm / amperage)
  print("Oops! Let's try that again...")
  print(">Actual voltage: ", ohm * amperage)
  print("...Good thing the computer double-checked!")

  print("\nNow, let's see if we can get the wattage too...")
  print(">Wattage: ", amperage * ohm**2)
  print("Square volts per amps? That's not right...")
  print(">Actual Wattage: ", amperage**2 * ohm)

  print("\nOne last thing. How much energy will this part use every second?")
  print("Energy: ", Measurement(1, 0, "s") / ohm)
  print("...farad!? That's a measure of capacitance, not power, that can't be right...")
  print("Energy: ", amperage**2 * ohm * Measurement(1, 0, "s"))
  print("There we go!")

test4()

>Voltage:  1692 ±169 ΩA⁻¹
Oops! Let's try that again...
>Actual voltage:  28.6 ±2.9 V
...Good thing the computer double-checked!

Now, let's see if we can get the wattage too...
>Wattage:  6292.000 ±0.026 V²A⁻¹
Square volts per amps? That's not right...
>Actual Wattage:  3.72 ±0.37 W

One last thing. How much energy will this part use every second?
Energy:  0.00455 ±0.00045 F
...farad!? That's a measure of capacitance, not power, that can't be right...
Energy:  3.72 ±0.37 J
There we go!
